In [ ]:
# | default_exp conversions_utils

In [ ]:
# | hide
from nbdev.showdoc import *
from fastcore.test import is_close

In [ ]:
# | export
import warnings
import operator
import collections
import numpy as np
from math import pi
from typing import Dict
from numpy import exp, cos, sin, arccos, arctan

In [ ]:
# | export


def convert_f_to_v(
    x,  # Soil value to be converted (m3.m-3)
    layer_thickness: float,  # Thickness of the soil layer (in m)
    rfc: float = 0,  # Rock fragment content of the soil layer (%)
) -> float:  # Soil water height (mm)
    "Convert soil volumetric water content (m3.m-3) to water height (mm) according to thickness and rock fragment content of the given soil layer"

    return x * (1 - (rfc / 100)) * layer_thickness * 1000

In [ ]:
# | export


def convert_vwc_to_sws(
    volumetric_water_content_x: float,  # Volumetric water content (m3.m-3)
    layer_thickness: float,  # Thickness of the soil layer (in m)
    rfc: float = 0,  # Rock fragment content of the soil layer (%)
) -> float:  # Soil Water Stock (mm)
    "Convert soil volumetric water content (VWC) to soil water stock (SWS). The volume of the water quantity per square metre results in the corresponding water stock height (m3 water per m2 soil as height in mm)"

    return (
        volumetric_water_content_x * (1 - (rfc / 100)) * layer_thickness * 1000
    )

In [ ]:
convert_vwc_to_sws(volumetric_water_content_x=0.5, rfc=50, layer_thickness=1)

250.0

In [ ]:
# | export


def convert_sws_to_vwc(
    soil_water_stock_x: float,  # Soil water stock (mm)
    layer_thickness: float,  # Thickness of the soil layer (in m)
    rfc: float = 0,  # Rock fraction  content of the soil layer (%)
) -> float:  # Volumetric Water Content (m3.m-3)
    "Convert soil water stock (SWS) to volumetric water content (VWC). Convert soil water stock (quantity as height in mm per m2 soil) to volumetric water content (m3.m-3) by accounting for the respective layer thickness and rock fragment content."
    return soil_water_stock_x / ((1 - (rfc / 100)) * layer_thickness * 1000)

In [ ]:
convert_sws_to_vwc(soil_water_stock_x=250, rfc=50, layer_thickness=1)

0.5

In [ ]:
# | export
def convert_ksat_cmday_to_ksat_micromol(
    ksat_cmday: float,  # Saturated hydraulic conductivity? in centimeters per day
) -> float:  # Saturated Hydraulic Conductivity?(ksat) in mmol/s/m/MPa
    "Convert Saturated Hydraulic Conductivity?(ksat) units from cm/day (ESDAC unit) to mmol/s/m/MPa (Sureau unit)"

    warnings.warn('Not sure if formula is correct')
    # Should it be (ksat_cmday/24)/(3600/100)
    ksat_ms = ksat_cmday / 24 / 3600 / 100

    ksat_kg_s_m3 = ksat_ms / 0.0098

    ksat_mmol_s_m_mpa = ksat_kg_s_m3 * 1000 * 1000 * 1000 / 18

    return ksat_mmol_s_m_mpa

In [ ]:
# | export
def convert_rate_to_molm_s_m_mpa(
    rate_x: float,  # A rate in L/T. The default assumes cm/d to be converted to m/s
    molm: float = 18.01528,  # Molar mass of h2o (g mol-1)
    pa: float = 9.80665,  # Hydrostatic pressure (Pa mm-1)
    xfac: float = 8640000,  # Divisor converting rate (L T-1) to (m s-1). Value 8640000 comes from 24 * 60 ** 2 * 100
) -> float:  # Converted rate to SurEau-Ecos unit (mmol.h2o s-1 m-1.soil MPa-1)
    "Convert a hydraulic rate to SurEau-Ecos rate"

    # TODO: consider temperature-dependency of water pressure? e.g.
    # pa <- getPa(t = 273.15 + 4) # find specific formula

    return rate_x * 1e12 / molm / pa / xfac

In [ ]:
convert_rate_to_molm_s_m_mpa(1)

655.1255924715483

In [ ]:
# | export
def convert_molm_s_m_mpa_to_rate(
    rate_x: float,  # # Unknown parameter definition
    molm: float = 18.01528,  # Molar mass of h2o (g mol-1)
    pa: float = 9.80665,  # Hydrostatic pressure (Pa mm-1)
    xfac: float = 8640000,  # Divisor converting rate (L T-1) to (m s-1). Value 8640000 comes from 24 * 60 ** 2 * 100
) -> float:  # A rate (L/T)
    "Convert SurEau-Ecos rate to a hydraulic rate. Inverse of convert_rate_to_molm_s_m_mpa() funtion."

    return rate_x / 1e12 * molm * pa * xfac

In [ ]:
convert_molm_s_m_mpa_to_rate(655.1255924715483)

0.9999999999999999

In [ ]:
# | export


def convert_mpa_to_m(
    mpa_x: float,  # Hydrostatic pressure in MPa
    pa: float = 9.80665,  # Hydrostatic pressure (Pa mm-1)
) -> float:  # Pressure height in meters
    "Conversion of hydrostatic pressure (MPa) to pressure height (m)"

    return mpa_x * 1e3 / pa

In [ ]:
convert_mpa_to_m(1)

101.97162129779284

In [ ]:
# | export
def convert_m_to_mpa(
    m_x: float,  # Pressure height in meters
    pa: float = 9.80665,  # Hydrostatic pressure (Pa mm-1)
) -> float:
    "Conversion of pressure height (m) to hydrostatic pressure (MPa)"

    return m_x / 1e3 * pa

In [ ]:
convert_m_to_mpa(101.97162129779284)

1.0

In [ ]:
# | export
def convert_pf_to_hpa(
    x: float,  # Unknown parameter definition
) -> float:  # Unknown parameter definition
    "Function lacks definition, documentation needs to be improved."

    return 10 ** abs(x)

In [ ]:
convert_pf_to_hpa(1.8)

63.09573444801933

In [ ]:
# | export
def convert_hpa_to_pf(
    x: float,  # Unknown parameter definition
) -> float:  # Unknown parameter definition
    "Function lacks definition, documentation needs to be improved."

    return np.log10(abs(x))

In [ ]:
convert_hpa_to_pf(63.09573444801933)

1.8

In [ ]:
# | export
def convert_vg_psi_to_theta(
    vg_psi_x: float,  # Soil matric potential (hPa)
    theta_s: float,  # Saturated water content (L3 L-3)
    theta_r: float,  # Residual water content (L3 L-3)
    a: float,  # Parameter related to the inverse of the air-entry pressure (unitless?)
    n: float,  # Parameter related to the pore-size distribution (unitless)
) -> float:  # Volumetric water content
    "Unit conversion from soil matric potential (Psi) to volumetric water content (Theta) using the Mualem-van-Genuchten pedotransfer function"

    return theta_r + (theta_s - theta_r) / (1 + abs(a * vg_psi_x) ** n) ** (
        1 - 1 / n
    )

In [ ]:
convert_vg_psi_to_theta(
    vg_psi_x=np.array([-1e6, -1e2, 0]), theta_s=0.5, theta_r=0.1, a=0.05, n=1.5
)

array([0.10178885, 0.27384953, 0.5       ])

In [ ]:
# | export


def convert_vg_theta_to_psi(
    vwc_theta_x: float,  # volumetric water content (L3 L-3)
    theta_s: float,  # Saturated water content (L3 L-3)
    theta_r: float,  # Residual water content (L3 L-3)
    a: float,  # Parameter related to the inverse of the air-entry pressure (unitless?)
    n: float,  # Parameter related to the pore-size distribution (unitless)
) -> float:  # Soil matric potential
    "Unit conversion from volumetric water content (Theta) to soil matric potential (Psi) using the Mualem-van-Genuchten pedotransfer function"

    theta_eff = (vwc_theta_x - theta_r) / (theta_s - theta_r)

    # *(-1) for negative psi
    return (theta_eff ** (-1 / (1 - 1 / n)) - 1) ** (1 / n) / a * (-1)

In [ ]:
convert_vg_theta_to_psi(
    vwc_theta_x=np.array([0.1, 0.3, 0.5]),
    theta_s=0.5,
    theta_r=0.1,
    a=0.05,
    n=1.5,
)

/tmp/ipykernel_9862/3671110304.py:16: RuntimeWarning: divide by zero encountered in power
  return (theta_eff ** (-1/(1 - 1/n)) - 1) ** (1/n) / a * (-1)


array([       -inf, -73.1861142,  -0.       ])

In [ ]:
# | export
def convert_vwc_to_rew(
    vwc_x: float,  # Volumetric Water Content (L3*L-3)
    theta_s: float,  # Saturated water content (L3 L-3)
    theta_r: float,  # Residual water content (L3 L-3)
) -> float:  # Relative Extractable water
    "Conversion from volumetric water content to relative extractable water (REW). REW is the volumetric water content normalised to saturation capacity (REW=1) and residual capacity (REW=0)"

    return (vwc_x - theta_r) / (theta_s - theta_r)

In [ ]:
convert_vwc_to_rew(vwc_x=np.array([0.1, 0.3, 0.5]), theta_s=0.5, theta_r=0.1)

array([0. , 0.5, 1. ])

In [ ]:
# | export
def convert_rew_to_vwc(
    rew_x: float,  # Relative Extractable water (rew)
    theta_s: float,  # Saturated water content (L3 L-3)
    theta_r: float,  #  Residual water content (L3 L-3)
) -> float:  # Volumetric Water Content (L3*L-3)
    "Conversion from relative extractable water to volumetric water"
    return rew_x * (theta_s - theta_r) + theta_r

In [ ]:
convert_rew_to_vwc(rew_x=np.array([0, 0.5, 1]), theta_s=0.5, theta_r=0.1)

array([0.1, 0.3, 0.5])

In [ ]:
# | export
def convert_mmol_to_m3(
    mmol_h2o_x: float,  # Unit to be converted (mmol, m³ or mm depending on the used function
    molm: float = 18.01528,  # Molar mass of h2o (g mol-1)
    pw: float = 998,  # Density of water (kg m-3.h2o)
) -> float:  # Unknown parameter
    "Stochiometric conversion between volume and molarity  In case of converting from or to millimetres (mm), the actual unit is the volume divided by its area, resulting in a depth given in millimetres."

    return mmol_h2o_x * molm / pw / 1e6  # 1e6 = mmol to mol and g to kg

In [ ]:
convert_mmol_to_m3(1)

1.8051382765531063e-08

In [ ]:
def convert_m3_to_mmol(
    m3_x: float,
    molm: float = 18.01528,  # Molar mass of h2o (g mol-1)
    pw: float = 998,  # Density of water (kg m-3.h2o)
) -> float:  #
    "Stochiometric conversion between volume and molarity  In case of converting from or to millimetres (mm), the actual unit is the volume divided by its area, resulting in a depth given in millimetres."

    return m3_x / molm * pw * 1e6

In [ ]:
convert_m3_to_mmol(1.8051382765531063e-08)

1.0

In [ ]:
# | export
def convert_mmol_to_mm(
    mmol_h2o_x: float,  # Unit to be converted (mmol, m³ or mm depending on the used function
    molm: float = 18.01528,  # Molar mass of h2o (g mol-1)
    pw: float = 998,  # Density of water (kg m-3.h2o)
) -> float:  # Unknown parameter
    "Stochiometric conversion between volume and molarity  In case of converting from or to millimetres (mm), the actual unit is the volume divided by its area, resulting in a depth given in millimetres."
    return mmol_h2o_x * molm / pw / 1e3  # 1e6 = mmol to mol and g to kg

In [ ]:
convert_mmol_to_mm(1)

1.8051382765531063e-05

In [ ]:
def convert_mm_to_mmol(
    mm_x: float,
    molm: float = 18.01528,  # Molar mass of h2o (g mol-1)
    pw: float = 998,  # Density of water (kg m-3.h2o)
) -> float:
    "Stochiometric conversion between volume and molarity  In case of converting from or to millimetres (mm), the actual unit is the volume divided by its area, resulting in a depth given in millimetres."

    return mm_x / molm * pw * 1e3

In [ ]:
convert_mm_to_mmol(1.8051382765531063e-05)

1.0

In [ ]:
# | export
def convert_compartment_leaf_to_soil(
    x: float,  # Unknown parameter definition m2.leaf
    lai: float = 1,  # Leaf Area Index
) -> float:  # Unknown parameter m2.soil
    "Function lacks definition, documentation needs to be improved."
    return x * lai

In [ ]:
# | export
def convert_compartment_soil_to_leaf(
    x: float,  # Unknown parameter definition m2.soil
    lai: float = 1,  # Leaf Area Index
) -> float:  # Unknown parameter m2.leaf
    "Function lacks definition, documentation needs to be improved."
    return x / lai

In [ ]:
# | export
def convert_compartment_stem_to_leaf(
    stem_x: float,  # Unknown parameter definition m2.stem
    hv: float,  # Huber value (sap wood / leaf area)
) -> float:  # Unknown parameter m2.leaf
    "Function lacks definition, documentation needs to be improved."
    return stem_x * hv

In [ ]:
# | export
def convert_compartment_leaf_to_stem(
    leaf_x: float,  # Unknown parameter definition m2.leaf
    hv: float,  # Huber value (sap wood / leaf area)
) -> float:  # Unknown parameter m2.stem
    "Function lacks definition, documentation needs to be improved."
    return leaf_x / hv

In [ ]:
# | export
def convert_compartment_stem_to_soil(
    stem_x: float,  # Unknown parameter definition
    swi: float,  # Sap wood index (m2.sapwood m-2.soil)
) -> float:  # Unknown parameter m2.soil
    "Function lacks definition, documentation needs to be improved."
    return stem_x * swi

In [ ]:
# | export
def convert_compartment_soil_to_stem(
    soil_x: float,  # Unknown parameter definition
    swi: float,  # Sap wood index (m2.sapwood m-2.soil)
) -> float:  # Unknown parameter m2.stem
    "Function lacks definition, documentation needs to be improved."

    return soil_x / swi

In [ ]:
def convert_stock_to_flux(
    x: float,
    dt: int,  # Time step length in seconds
) -> float:  #
    "Function lacks definition, documentation needs to be improved."
    return x / dt

In [ ]:
def convert_flux_to_stock(
    x: float,
    dt: int,  # Time step length in seconds
) -> float:  #
    "Function lacks definition, documentation needs to be improved."
    return x * dt

In [ ]:
# | hide
# def convert_flux_mmol_s_m2_leaf_to_mm_m2_soil(
#    mmol_h2o_m2_leaf_s_1_x: float,  # Unknown parameter definition
#    lai: float,  #
#    dt: float,  # Time step length in seconds
# ) -> float:  # Unknown parameter definition mm.h2o m-2.soil h-1 dt-1
#
#    'Function lacks definition, documentation needs to be improved.'
#
#    x_m2_soil = convert_flux_m2_leaf_to_m2_soil(
#        m2_leaf_x=mmol_h2o_m2_leaf_s_1_x, lai=lai
#    )
#    x_m3 = convert_flux_mmol_to_m3(mmol_h2o_x=x_m2_soil)
#
#    # 1e3 = m3 to l = mm m-2.soil
#    return x_m3 * 1e3 * dt * 3600

In [ ]:
# | hide
# def convert_flux_mmol_s_m2_leaf_to_mm_m2_stem(
#    mmol_h2o_m2_leaf_s_1_x: float,  # Unknown parameter definition
#    hv: float,  # Hubber value?
#    dt: float,  # Unknown parameter definition
# ) -> float:  # mm.h2o m-2.stem h-1 dt-1
#    'Function lacks definition, documentation needs to be improved.'
#
#    x_m2_stem = convert_flux_m2_leaf_to_m2_stem(
#        m2_leaf_x=mmol_h2o_m2_leaf_s_1_x, hv=hv
#    )
#    x_m3 = convert_flux_mmol_to_m3(mmol_h2o_x=x_m2_stem)
#
#    # 1e3 = m3 to l = mm m-2.soil
#    return x_m3 * 1e3 * dt * 3600

In [ ]:
# | export
def convert_stock_mm_soil_to_flux_mmol_leaf(
    stand_water_stock_x: float,  # Stand level water stock for a given LAI in mm(L m-2Soil) over the period of time considered
    lai: float,  # Leaf area Index
    dt: float,  # Time step length in seconds
    molm: float = 18.01528,  # Molar mass of h2o (g mol-1)
    pw: float = 998,  # Density of water (kg m-3.h2o)
) -> float:  # Instantaneous leaf area-based flux (mmol m-2Leaf s-1)
    "Converts a stand level water quantity (mm, i.e. L m-2Soil) over a defined time period to a leaf area-based flux (mmol m-2Leaf s-1) for a given leaf area index"

    if isinstance(stand_water_stock_x, float):
        # Transform floatg values into array
        stand_water_stock_x = np.array([stand_water_stock_x])

    # Conversion ----------------------------------------------------------------
    if lai > 0:
        flux = convert_stock_to_flux(x=stand_water_stock_x, dt=dt)
        mmol = convert_mm_to_mmol(mm_x=flux, molm=molm, pw=pw)

        return convert_compartment_soil_to_leaf(x=mmol, lai=lai)

    else:
        # print(len(np.array(stand_water_stock_x)))
        return np.repeat(0, repeats=len(stand_water_stock_x))

In [ ]:
convert_stock_mm_soil_to_flux_mmol_leaf(stand_water_stock_x=2, dt=60**2, lai=3)

10.25878114660526

In [ ]:
def convert_flux_mmol_leaf_to_stock_mm_soil(
    leaf_area_based_flux: float,  # Instantaneous leaf area-based flux to be converted (mmol m-2Leaf s-1)
    dt: float,  # Time step length in seconds
    lai: float,  # Leaf area Index
    molm: float = 18.01528,  # Molar mass of h2o (g mol-1)
    pw: float = 998,  # Density of water (kg m-3.h2o)
) -> float:  # Stand level water stock for a given LAI in mm (L m-2Soil) over the period of time considered.
    "Converts an instantaneous leaf area-based flux (mmol m-2Leaf s-1) to a stand level water quantity (mm, i.e. L m-2Soil) for a given leaf area index over a defined time period."
    stock = convert_flux_to_stock(x=leaf_area_based_flux, dt=dt)
    mm = convert_mmol_to_mm(mmol_h2o_x=stock, molm=molm, pw=pw)

    return convert_compartment_leaf_to_soil(x=mm, lai=lai)

In [ ]:
# 10 mmol/s over a period of 1 hour with an LAI of 3 m² is converted to
# millimetres of soil water stock (L/m²)

convert_flux_mmol_leaf_to_stock_mm_soil(leaf_area_based_flux=10, dt=60**2, lai=3)

1.9495493386773552

In [ ]:
# | export
def convert_optim_mm_soil_to_mmol_leaf(
    instaneous_flux: float,  # Stand level water stock for a given LAI in mm (L m-2Soil) over the period of time considered
    dt: float,  # Time step length in seconds
    lai: float,  # Leaf area index
) -> float:  # Instantaneous leaf area-based flux (mmol m-2Leaf s-1
    "Converts a stand level water quantity (mm, i.e. L m-2Soil) over a defined time period to a leaf area-based flux (mmol m-2Leaf s-1) for a given leaf area index."
    if lai > 0:
        # originally version
        # x * LAI * dt * 3600 * 18 / 1e6
        #
        # hard-coded unit conversion for computational speed-up
        # - 3600 * 18 / 1e6 = 0.0648
        # - TODO: more exact and equally fast would be
        #   hours_to_secs * molm / pw / 1e3
        #   3600 * 18.01528 / 998 / 1e3 = 0.06498498
        # a more flexible function is given with convert_flux_mmol_s_m2Leaf_To_mm_m2Soil
        return instaneous_flux / lai * dt * 1.8e-5  # 0.06498498

    else:
        raise ValueError('Failed convert_optim_mm_soil_to_mmol_leaf. lai < 0')

In [ ]:
# | export
def convert_optim_mmol_leaf_to_mm_soil(
    x: float,  # Instantaneous leaf area-based flux to be converted (mmol m-2Leaf s-1)
    dt: float,  # Time step length in seconds
    lai: float,  # Leaf area index
) -> float:  # Stand level water stock for a given LAI in mm (L m-2Soil) over the period of time considered
    "Converts an instantaneous leaf area-based flux (mmol m-2Leaf s-1) to a stand level water quantity"

    return x * lai * dt * 1.8e-5

## Functions of conversion and temporal disagregation of climatic variable as well as PET computation



In [ ]:
# | export
def compute_vpd_from_temp_rh(
    relative_humidity: float,  # Air relative_humidity (%)
    temperature: float,  # Air temperature (degrees Celsius)
    air_pressure: float = 101325,  # Air pressure (Pa)
) -> float:
    "Compute vapor pressure deficit (VPD) from air relative humidity (rh) and air temperature (temp)"

    # Assert parameters ---------------------------------------------------------

    # relative_humidity

    # Using np.testing instead of assert because parameters can be np.arrays OR
    # single values (i.e. 1). assert only works when params are always one type
    # Solution from:
    # https://stackoverflow.com/questions/45987962/why-arent-there-numpy-testing-assert-array-greater-assert-array-less-equal-as

    np.testing.assert_array_compare(
        operator.__gt__,
        np.array(relative_humidity),
        0,
        err_msg='\nrelative_humidity must be must be a integer value between 0-100\n',
    )

    np.testing.assert_array_less(
        np.array(relative_humidity),
        101,
        err_msg='\nrelative_humidity must be must be a integer value between 0-100\n',
    )

    # temperature
    np.testing.assert_array_compare(
        operator.__gt__,
        np.array(temperature),
        -100,
        err_msg='\ntemperature must be must be value between -100 and 100\n',
    )

    np.testing.assert_array_less(
        np.array(temperature),
        101,
        err_msg='\ntemperature must be must be value between -100 and 100\n',
    )

    # Define Constants ----------------------------------------------------------

    # molar weight dry air (g/mol)
    molar_weight_dry_air = 28.966

    # molar weight H20 H2O(g/mol) Mh2o not used in this function??
    # mass_h2o = 18

    # Perfect gas constant %J/mol/K
    rgz = 8.314472

    # conversion of temperature in K
    temp_kelvin = temperature + 273.15

    # D_air not used in this function??
    # d_air = ((air_pressure) / (rgz * (temp_kelvin))) * molar_weight_dry_air

    # Using esat calculation from Planyecophys R package
    a = 611.21
    b = 17.502
    c = 240.97
    f = 1.0007 + 3.46 * 1e-08 * 101 * 1000

    es = f * a * (exp(b * temperature / (c + temperature)))

    # Old calculation
    # es = 6.108 * exp(17.27 * temperature / (237.2 + temperature)) * 100

    ea = relative_humidity / 100 * es

    # Compute VPD ---------------------------------------------------------------

    vpd = (es - ea) / 1000

    # Step implemented in case vpd is a single value i.e. vpd = 41
    if isinstance(vpd, float):
        # Transform floatg values into array
        vpd = np.array([vpd])

    # else not included because generates erro

    # else:
    #    raise ValueError(
    #            'Error computing vpd from rh and temp'
    #        )

    # Remove negative values if exits
    vpd[vpd < 0] = 0

    return vpd

#### __Example: Compute VPD__

In [ ]:
compute_vpd_from_temp_rh(relative_humidity=70, temperature=55)

array([4.76019573])

In [ ]:
compute_vpd_from_temp_rh(relative_humidity=25, temperature=np.array([80, 90]))

array([36.10624792, 53.70318944])

In [ ]:
# | hide

# Test that function return a positive value
assert compute_vpd_from_temp_rh(relative_humidity=80, temperature=25) > 0

In [ ]:
# | hide

# Test that function return a known anwser
is_close(
    compute_vpd_from_temp_rh(relative_humidity=80, temperature=25),
    # Value from https://cales.arizona.edu/vpdcalc/
    0.634,
    eps=1e-02,
)

True

In [ ]:
# | export
def compute_slope_sat(
    tmoy: float,  # Mean daily temperature in °C
) -> float:  # Slope of the saturation vapour pressure function
    "Compute slope of the saturation vapour pressure function (AO 1998)"

    return (
        4098
        * 0.6108
        * np.exp((17.27 * tmoy) / (tmoy + 237.3))
        / ((tmoy + 237.3) ** 2)
    )

In [ ]:
# | export


def compute_pet(
    tmoy: float,  # Mean temperature over the considered time step (degrees Celsius)
    net_radiation: float,  # Cumulative Net radiation over the considered time sep (MJ.m2)
    g: float = 0,  # Energy storage parameter. Represents ground heat flux
    vpd: float = None,  # Vapor pressure deficit (kpa) for calculating etp using the Penman-Monteith (penman) formulation
    wind_speed_u: float = None,  #  Wind speed (m.s-1) for calculating etp using the Penman-monteith (penman) formulation
    pt_coeff: float = None,  # An empirical constant accounting for the vapor pressure deficit and resistance values. Typically, α is 1.26 for open bodies of water, but has a wide range of values from less than 1 (humid conditions) to almost 2 (arid conditions).
    formulation: str = 'pt',  # String indicating which formulation to use (Pristeley Taylor (pt) or Penman-Monteith (penman)) for calculating potential evapotranspiration
) -> float:  #  Potential evapotranspiration (PET) (mm)
    "Calcule Potential Evapotranspiration (mm) PET using Pristeley Taylor (pt) or Penmman (penman) formulation"

    # Assert parameters ---------------------------------------------------------
    # tmoy
    assert (
        isinstance(tmoy, float)
        | isinstance(tmoy, int)
        | isinstance(tmoy, np.ndarray)
    ), 'tmoy parameter must be a float or integer value'

    # net_radiation
    assert (
        isinstance(net_radiation, float)
        | isinstance(net_radiation, int)
        | isinstance(net_radiation, np.ndarray)
    ), 'net_radiation parameter must be a float or integer value'

    # g
    assert (
        isinstance(g, float) | isinstance(g, int) | isinstance(g, np.ndarray)
    ), 'g parameter must be a float or integer value'

    # Formulation
    assert formulation in ['pt', 'penman'], (
        f'{formulation} not a valid option for formulation, select "pt" for Pristeley Taylor (pt) or "penman" for Penmman'
    )

    # wind_speed
    if formulation == 'penman':
        assert (
            isinstance(wind_speed_u, float)
            | isinstance(wind_speed_u, int)
            | isinstance(wind_speed_u, np.ndarray)
        ), (
            'Parameter wind_speed_u required for penman formulation. This must be a float or integer value'
        )

        # vpd
        assert (
            isinstance(vpd, float)
            | isinstance(vpd, int)
            | isinstance(vpd, np.ndarray)
        ), (
            'Parameter vpd required for penman formulation. This must be a float or integer value'
        )

    # pt_coeff
    if formulation == 'pt' and pt_coeff is None:
        warnings.warn('Using default value of 1.26 for pt_coeff')
        pt_coeff = 1.26

    elif formulation == 'pt' and pt_coeff is not None:
        assert (
            isinstance(pt_coeff, float)
            | isinstance(pt_coeff, int)
            | isinstance(pt_coeff, np.ndarray)
        ), (
            'Parameter pt_coeff required for pt formulation. This must be a float or integer value'
        )

    # Calculate pet -------------------------------------------------------------

    # Pristeley-Taylor formulation
    if formulation == 'pt':
        # Define Constants

        # Stefan-Boltzman constant [MJ.K^-4.m^-2.day^-1]
        # sb_constant = 4.903 * 10**9

        # Psychometer constant
        gamma = 0.0666

        # Latent heat of vaporisation
        lamb = 2.45

        #  s: slope of the saturation vapour pressure function (AO 1998)
        slope_sta = compute_slope_sat(tmoy=tmoy)

        # Return ETP
        return (
            pt_coeff
            * (slope_sta / (slope_sta + gamma))
            * ((net_radiation - g) / lamb)
        )

    # Penman formulation
    elif formulation == 'penman':
        # Define Constants

        # Stefan-Boltzman constant [MJ.K^-4.m^-2.day^-1]
        # sb_constant = 4.903 * 10**9

        # Psychometer constant
        gamma = 0.0666

        # Latent heat of vaporisation
        # lambda = 2.45

        #  s: slope of the saturation vapour pressure function (AO 1998)
        delta = (
            4098
            * 0.6108
            * np.exp((17.27 * tmoy) / (tmoy + 237.3))
            / ((tmoy + 237.3) ** 2)
        )

        # ga = 0.34 * max(u, 0.001)

        u2 = wind_speed_u * (4.87 / np.log(67.8 * 10 - 5.42))

        n1 = 0.408 * delta * net_radiation
        n2 = gamma * (37 / (tmoy + 273)) * u2 * vpd
        d = delta + gamma * (1 + 0.34 * u2)

        # Return pet
        return (n1 + n2) / (d)

#### __Example: Compute Potential Evapotranspiration__

In [ ]:
compute_pet(tmoy=80, net_radiation=25, formulation='pt')

In [ ]:
compute_pet(
    tmoy=20, net_radiation=1, wind_speed_u=2, vpd=2, formulation='penman'
)

In [ ]:
compute_pet(
    tmoy=15, net_radiation=5, wind_speed_u=2, vpd=1, formulation='penman'
)

1.126055124868983

In [ ]:
# | hide

# Test that function return a positive value
assert compute_pet(tmoy=80, net_radiation=25, formulation='pt') > 0

In [ ]:
# | hide

# Test that function return a known anwser
is_close(
    compute_pet(tmoy=80, net_radiation=25, formulation='pt'), 12.4292, eps=1e-02
)

In [ ]:
# | hide
# Test that function return a positive value
assert (
    compute_pet(
        tmoy=20, net_radiation=1, wind_speed_u=2, vpd=2, formulation='penman'
    )
    > 0
)

In [ ]:
# | hide

# Test that function return a known anwser
is_close(
    compute_pet(
        tmoy=20, net_radiation=1, wind_speed_u=2, vpd=2, formulation='penman'
    ),
    0.3434,
    eps=1e-02,
)

In [ ]:
# | export


def calculate_radiation_diurnal_pattern(
    time_of_day: int,  # Numeric value of vector indicating the time of the day (in seconds)
    day_length: int,  # Value indicating the duration of the day (in seconds). Calculated using the `day_length` function
) -> float:
    "Calculated diurnal pattern of temperature assuming a sinusoidal pattern with T = tmin at sunrise and T = (tmin + tmax)/2 at sunset. From sunset to sunrise follows a linear trend"

    # Assert parameters ---------------------------------------------------------

    # time_of_day

    # Using np.testing instead of assert because parameters can be np.arrays OR
    # single values (i.e. 1). assert only works when params are always one type
    # Solution from:
    # https://stackoverflow.com/questions/45987962/why-arent-there-numpy-testing-assert-array-greater-assert-array-less-equal-as

    # if time_of_day < 0:
    #    warnings.warn(
    #        'time_of_day is a negative value. Not sure if this is correct'
    #    )
    # np.testing.assert_array_compare(
    #    operator.__ge__,
    #    np.array(time_of_day),
    #    0,
    #    err_msg="\ntime_of_day must be must be value between 0 and 86400\n",
    # )
    #
    # np.testing.assert_array_less(
    #    np.array(time_of_day),
    #    86401,
    #    err_msg="\ntime_of_day must be must be value between 0 and 86400\n",
    # )

    # day_length
    np.testing.assert_array_compare(
        operator.__ge__,
        np.array(day_length),
        0,
        err_msg='\nday_length must be must be value between 0 and 86400\n',
    )

    np.testing.assert_array_less(
        np.array(day_length),
        # This value is 86401 and not 86400 because of the
        # np.testing.assert_array_less
        86401,
        err_msg='\nday_length must be must be value between 0 and 86400\n',
    )

    # Raise warning for time_of_day and day_length
    # warnings.warn('Double check that time_of_day and day_length are in seconds')

    # Calculate_radiation_diurnal_pattern ---------------------------------------

    # sunrise
    ws = (day_length / 3600.0) * (pi / 24.0)
    w = ws - (time_of_day / day_length) * (ws * 2.0)

    prop = ((pi / 24.0) * (cos(w) - cos(ws))) / (sin(ws) - ws * cos(ws))

    return prop / 3600.0

#### __Example: Calculate radiation diurnal pattern__

In [ ]:
calculate_radiation_diurnal_pattern(time_of_day=1, day_length=40)

In [ ]:
# | export


def calculate_temperature_diurnal_pattern(
    time_of_day: float,  # Value of vector indicating the time of the day (in seconds from sunrise)
    day_length: int,  # Value indicating the duration of the day (in seconds). Calculated using the `day_length` function
    tmin: float,  # Minimum temperature (in degrees C) of the target day of the year
    tmax: float,  # Maximum temperature (in degrees C) of the target day of the year
    tmin_prev: float,  # Minimum temperature (in degrees C) of the previous target day of the year
    tmax_prev: float,  # Maximum temperature (in degrees C) of the previous target day of the year
    tmin_next: float,  # Minimum temperature (in degrees C) of the next target day of the year
) -> float:  # Temperature at the given time
    "Calculated diurnal pattern of temperature assuming a sinusoidal pattern with T = tmin at sunrise and T = (tmin+tmax)/2 at sunset. From sunset to sunrise follows a linear trend"

    # Raise warning for time_of_day and day_length
    warnings.warn('Double check that time_of_day and day_length are in seconds')

    # calculate_temperature_diurnal_pattern -------------------------------------

    if time_of_day < 0.0 or time_of_day >= day_length:
        tfin = 86400.0 - day_length

        if time_of_day < 0.0:
            time_of_day = time_of_day + 86400.0 - day_length

            # Return Temp
            return 0.5 * (tmax_prev + tmin_prev) * (
                1.0 - (time_of_day / tfin)
            ) + tmin * (time_of_day / tfin)

        else:
            time_of_day = time_of_day - day_length

            # Return Temp
            return 0.5 * (tmax + tmin) * (
                1.0 - (time_of_day / tfin)
            ) + tmin_next * (time_of_day / tfin)

    else:
        ct = cos(1.5 * pi * time_of_day / day_length)

        # Return Temp
        return 0.5 * (tmin + tmax - (tmax - tmin) * ct)

#### __Example: calculate the temperature diurnal pattern__

In [ ]:
calculate_temperature_diurnal_pattern(
    time_of_day=0,
    day_length=0,
    tmin=0.6,
    tmax=8.2,
    tmin_prev=2.6,
    tmax_prev=11.3,
    tmin_next=4.7,
)

In [ ]:
# | export


def calculate_rh_diurnal_pattern(
    temperature: float,  # Numeric value providing the target temperature (degrees C) at which we should compute target rh
    rhmin: float,  # Numeric value indicating the minimum relative humidity (%) of the day
    rhmax: float,  # Numeric value indicating the maximum relative humidity (%) of the day
    tmin: float,  # Numeric value indicating the minimum daily temperature (degrees C)
    tmax: float,  # Numeric value indicating the maximum daily temperature (degrees C)
) -> float:  # Relative humidity at given temperature
    "Calculate diurnal pattern of relative humidity from temperature"

    # Assert parameters ---------------------------------------------------------
    # rhmin
    assert (
        isinstance(rhmin, float) | isinstance(rhmin, int) and 100 >= rhmin >= 0
    ), 'Minimum relative humidity (rhmin) must be a value between 0 and 100'

    # rhmax
    assert (
        isinstance(rhmax, float) | isinstance(rhmax, int) and 100 >= rhmax >= 0
    ), 'Maximum relative humidity (rhmax) must be a value between 0 and 100'

    # calculate rh diurnal pattern ----------------------------------------------
    return rhmax + ((temperature - tmin) / (tmax - tmin)) * (rhmin - rhmax)

In [ ]:
calculate_rh_diurnal_pattern(17, 40, 80, 15, 25)

72.0

In [ ]:
# | export


def global_radiation_conversions_watts_ppfd(
    rg_ppfd: float = None,  # Photosynthetic photon flux density (umol.m-2.s-1)
    rg_watts: float = None,  # Global radiation (W/m2)
    j_to_mol: float = 4.6,  # Conversion factor
    frac_par: float = 0.5,  # Fraction of solar radiation that is photosynthetically active radiation (PAR)
    selected_conversion: str = 'rg_watts_to_rg_ppfd_umol',  # String indicating to what units rg should be converted. Options 'rg_ppfd_umol_to_rg_watts' or 'rg_watts_to_rg_ppfd_umol'
) -> float:
    "Convert Global Radiation (rg) in watts to Photosynthetic Photon Flux Density (ppfd) in umol and viceversa"

    # Assert parameters ---------------------------------------------------------

    # Make sure that selected_conversion only has three options
    assert selected_conversion in [
        'rg_ppfd_umol_to_rg_watts',
        'rg_watts_to_rg_ppfd_umol',
    ], (
        f'{selected_conversion} not a valid option for selected_conversion, select "rg_ppfd_umol_to_rg_watts" or "rg_watts_to_rg_ppfd_umol"'
    )

    # Make sure the necessary parameters for a given conversion are provided
    if selected_conversion == 'rg_ppfd_umol_to_rg_watts':
        assert (
            isinstance(rg_ppfd, float)
            | isinstance(rg_ppfd, int)
            | isinstance(rg_ppfd, np.ndarray)
        ), 'rg_ppfd missing. Parameter must be a float or integer value'

    elif selected_conversion == 'rg_watts_to_rg_ppfd_umol':
        assert (
            isinstance(rg_watts, float)
            | isinstance(rg_watts, int)
            | isinstance(rg_watts, np.ndarray)
        ), 'rg_watts missing. Parameter must be a float or integer value'

    # Warn in case j_to_mol or frac_par are not provided
    if j_to_mol == 4.6:
        warnings.warn('Using j_to_mol default value of 4.6')

    if frac_par == 0.5:
        warnings.warn('Using frac_par default value of 0.5')

    # Conversions ---------------------------------------------------------------

    # Calculate rg from ppfd (rg)(W/m2)
    if selected_conversion == 'rg_ppfd_umol_to_rg_watts':
        print('Global radiation (rg) converted from ppfd to watts')
        return rg_ppfd / frac_par / j_to_mol

    # Calculate ppfd (umol.m-2.s-1) from rg
    elif selected_conversion == 'rg_watts_to_rg_ppfd_umol':
        print('Global radiation (rg) converted from watts to ppfd')
        return rg_watts * frac_par * j_to_mol

    else:
        raise ValueError('Global Radiation conversion failed')

#### __Example: Calculate Global radiation in Watts and in PPFD__

In [ ]:
global_radiation_conversions_watts_ppfd(
    rg_watts=7,
    j_to_mol=4.6,
    frac_par=0.5,
    selected_conversion='rg_watts_to_rg_ppfd_umol',
)

Global radiation (rg) converted from watts to ppfd


/tmp/ipykernel_613/1345094339.py:41: UserWarning: Using j_to_mol default value of 4.6
  warnings.warn('Using j_to_mol default value of 4.6')
/tmp/ipykernel_613/1345094339.py:44: UserWarning: Using frac_par default value of 0.5
  warnings.warn('Using frac_par default value of 0.5')


16.099999999999998

In [ ]:
global_radiation_conversions_watts_ppfd(
    rg_ppfd=9,
    j_to_mol=4.6,
    frac_par=0.5,
    selected_conversion='rg_ppfd_umol_to_rg_watts',
)

Global radiation (rg) converted from ppfd to watts


/tmp/ipykernel_10557/2430486604.py:40: UserWarning: Using j_to_mol default value of 4.6
  warnings.warn('Using j_to_mol default value of 4.6')
/tmp/ipykernel_10557/2430486604.py:43: UserWarning: Using frac_par default value of 0.5
  warnings.warn('Using frac_par default value of 0.5')


3.91304347826087

In [ ]:
global_radiation_conversions_watts_ppfd(
    rg_watts=5,
    j_to_mol=4.6,
    frac_par=0.5,
    selected_conversion='rg_watts_to_rg_ppfd_umol',
)

Global radiation (rg) converted from watts to ppfd


/tmp/ipykernel_10557/2430486604.py:40: UserWarning: Using j_to_mol default value of 4.6
  warnings.warn('Using j_to_mol default value of 4.6')
/tmp/ipykernel_10557/2430486604.py:43: UserWarning: Using frac_par default value of 0.5
  warnings.warn('Using frac_par default value of 0.5')


11.5

In [ ]:
# | export


def global_radiation_conversions_watts_mjules(
    rg_watts: float = None,  # Instantaneous radiation (watts)
    rg_mj: float = None,  # Instantaneous radiation (in Mega Jules?)
    nhours: float = None,  # Number of hours of sun in the day
    selected_conversion: str = 'rg_watts_to_rg_mj',  # String indicating to what units rg should be converted. Options are 'rg_watts_to_rg_mj', 'rg_mj_to_rg_watts', 'rg_mj_to_rg_watts_hour', 'rg_watts_to_rg_mjday' or 'rg_mjday_to_rg_watts'
) -> float:
    "Convert instantaneous radiation in watts to daily cumulative radiation in MJ (MJ.day-1) and viceversa"

    # Assert parameters ---------------------------------------------------------

    # Make sure that selected_conversion only has three options
    assert selected_conversion in [
        'rg_watts_to_rg_mj',
        'rg_mj_to_rg_watts',
        'rg_mj_to_rg_watts_hour',
        'rg_watts_to_rg_mjday',
        'rg_mjday_to_rg_watts',
    ], (
        f'{selected_conversion} not a valid option for selected_conversion, select "rg_watts_to_rg_mj","rg_mj_to_rg_watts", "rg_mj_to_rg_watts_hour", "rg_watts_to_rg_mjday" or "rg_mjday_to_rg_watts"'
    )

    # Make sure the necessary parameters for a given conversion are provided
    if (
        selected_conversion == 'rg_watts_to_rg_mj'
        or selected_conversion == 'rg_watts_to_rg_mjday'
    ):
        assert (
            isinstance(rg_watts, float)
            | isinstance(rg_watts, int)
            | isinstance(rg_watts, np.ndarray)
        ), 'rg_watts missing. Parameter must be a float or integer value'

    elif (
        selected_conversion == 'rg_mj_to_rg_watts'
        or selected_conversion == 'rg_mjday_to_rg_watts'
    ):
        assert (
            isinstance(rg_mj, float)
            | isinstance(rg_mj, int)
            | isinstance(rg_mj, np.ndarray)
        ), 'rg_mj missing. Parameter must be a float or integer value'

    elif selected_conversion == 'rg_mj_to_rg_watts_hour':
        assert (
            isinstance(rg_mj, float)
            | isinstance(rg_mj, int)
            | isinstance(rg_mj, np.ndarray)
        ), 'rg_mj missing. Parameter must be a float or integer value'

        assert (
            isinstance(nhours, float)
            | isinstance(nhours, int)
            | isinstance(nhours, np.ndarray)
        ), 'nhours missing. Parameter must be a float or integer value'

    # Conversions ---------------------------------------------------------------
    if selected_conversion == 'rg_watts_to_rg_mj':
        print('Global radiation (rg) converted from watts to Mega Jules')

        # Conversion from watts to Mega Jules
        return rg_watts * 0.0864

    elif selected_conversion == 'rg_mj_to_rg_watts':
        print('Global radiation (rg) converted from Mega Jules to Watts')

        # Conversion from Mega Jules to watts
        return rg_mj * (1 / 0.0864)

    elif selected_conversion == 'rg_mj_to_rg_watts_hour':
        print(
            'Global radiation (rg) converted from Mega Jules to Watts per hour'
        )

        # Conversion from Mega Jules to watts/hour
        return rg_mj * (1e6 / (nhours * 3600))

    elif selected_conversion == 'rg_watts_to_rg_mjday':
        print('Global radiation (rg) converted from watts to MegaJules per day')

        # Conversion from watts to MegaJules day
        return rg_watts * 3600 * 24 / 1000000

    elif selected_conversion == 'rg_mjday_to_rg_watts':
        print('Global radiation (rg) converted from watts to MegaJules per day')

        # Conversion from MegaJules day to watts
        rg_mj / (3600 * 24 / 1000000)

    else:
        raise ValueError('Global radiation (rg) units conversion failed')

#### __Example: Calculate Global radiation in Watts and in Mega Jules__

In [ ]:
global_radiation_conversions_watts_mjules(
    rg_watts=300, selected_conversion='rg_watts_to_rg_mjday'
)

Global radiation (rg) converted from watts to MegaJules per day


25.92

In [ ]:
global_radiation_conversions_watts_mjules(
    rg_mj=25.92, selected_conversion='rg_mj_to_rg_watts'
)

Global radiation (rg) converted from Mega Jules to Watts


300.0

In [ ]:
global_radiation_conversions_watts_mjules(
    rg_mj=np.array([1, 2]),
    nhours=1,
    selected_conversion='rg_mj_to_rg_watts_hour',
)

Global radiation (rg) converted from Mega Jules to Watts per hour


array([277.77777778, 555.55555556])

In [ ]:
global_radiation_conversions_watts_mjules(
    rg_mj=10, nhours=10, selected_conversion='rg_mj_to_rg_watts_hour'
)

Global radiation (rg) converted from Mega Jules to Watts per hour


277.77777777777777

In [ ]:
# | export


def declination(
    day_of_year: int,  # julian day (day of the year)
    day_of_spring: int = 80,  # Julian day representing the first day of spring
) -> float:  # Earth declination at day_of_year
    "Calculate declination of sun (radians ? ) for a given julian day (DOY)"

    # Hervé's formula for solar declination

    # Sin(23.5*pi/180), 23.5 = Earth declination

    # Assert parameters ---------------------------------------------------------

    # Day of year
    # Using np.testing instead of assert because parameters can be np.arrays OR
    # single values (i.e. 1). assert only works when params are always one type
    # Solution from:
    # https://stackoverflow.com/questions/45987962/why-arent-there-numpy-testing-assert-array-greater-assert-array-less-equal-as

    np.testing.assert_array_compare(
        operator.__gt__,
        np.array(day_of_year),
        0,
        err_msg='\nday_of_year must be must be a integer value between 1-366\n',
    )

    np.testing.assert_array_less(
        np.array(day_of_year),
        367,
        err_msg='\nError: day_of_year must be must be a integer value between 1-366\n',
    )

    # date_of_spring
    assert isinstance(day_of_spring, int), (
        'day_of_spring must be must be a integer value i.e. 80, 90, 1'
    )

    # Constans ------------------------------------------------------------------

    c1 = 0.398749068925246

    c2 = 2 * 3.1416 / 365

    # date of spring
    day_of_spring_c3 = day_of_spring
    warnings.warn(
        f'date of spring set to {day_of_spring_c3}. This might change for Australia'
    )

    x = c1 * sin((day_of_year - day_of_spring_c3) * c2)

    # Return declination --------------------------------------------------------
    return arctan(x / ((1 - x * x) ** 0.5))

#### __Example: Calculate declination__

In [ ]:
declination(day_of_year=90, day_of_spring=80)

/tmp/ipykernel_613/1608233118.py:48: UserWarning: date of spring set to 80. This might change for Australia
  warnings.warn(


0.06835636102670284

In [ ]:
declination(day_of_year=np.array([100, 200]), day_of_spring=80)

/tmp/ipykernel_613/1608233118.py:48: UserWarning: date of spring set to 80. This might change for Australia
  warnings.warn(


array([0.1349969 , 0.35853539])

In [ ]:
# | export


def potential_par(
    time_of_day_in_hours: float,  # Array containing the time of the day (in hours) for which potential par should be calculated
    latitude: float,  # Numeric value specifying the geographic latitude (in decimal degrees) of the location of interest
    day_of_year: int,  # Julian day (day of the year)
) -> np.array:  # Potential Photosynthetic Active Radiation (PAR) for each time_of_day at given latitude and given day_of_year
    "Determine potential for a given place and date /used to determine cloud cover return potential par in W.m2"

    warnings.warn('Make sure time of day is hours in potential_par function')

    # Assert parameters ---------------------------------------------------------

    # time_of_day
    np.testing.assert_array_compare(
        operator.__ge__,
        np.array(time_of_day_in_hours),
        0,
        err_msg='\ntime_of_day must be must equal or greater than 0\n',
    )

    np.testing.assert_array_less(
        np.array(time_of_day_in_hours),
        25,
        err_msg='\nError: time_of_day must be must be equal or lower than 24\n',
    )

    # Latitude
    assert (
        isinstance(latitude, float) | isinstance(latitude, int)
        and 95 >= latitude >= -95
    ), (
        'Provide latitude as coordinates points bewteen -90 and 90 i.e. latitude = 41.40338'
    )

    # Day of year
    # Using np.testing instead of assert because parameters can be np.arrays OR
    # single values (i.e. 1). assert only works when params are always one type
    # Solution from:
    # https://stackoverflow.com/questions/45987962/why-arent-there-numpy-testing-assert-array-greater-assert-array-less-equal-as

    np.testing.assert_array_compare(
        operator.__gt__,
        np.array(day_of_year),
        0,
        err_msg='\nday_of_year must be must be a integer value between 1-366\n',
    )

    np.testing.assert_array_less(
        np.array(day_of_year),
        367,
        err_msg='\nError: day_of_year must be must be a integer value between 1-366\n',
    )

    # Calculate declination -----------------------------------------------------
    decl = declination(day_of_year=day_of_year, day_of_spring=80)

    # Define constants ----------------------------------------------------------
    diffuse_fraction = 0.1
    solar_constant = 2084
    attenuation_coef = -0.174353387144778

    pn = -cos(latitude * 3.1416 / 180)

    pz = sin(latitude * 3.1416 / 180)

    h_rad = (time_of_day_in_hours - 6) * 3.1416 / 12

    se = cos(h_rad) * cos(decl)
    sn = -pz * sin(h_rad) * cos(decl) - pn * sin(decl)
    sz = -pn * sin(h_rad) * cos(decl) + pz * sin(decl)

    alt = arctan(sz / ((se * se + sn * sn) ** 0.5))
    azi = 3.1416 + arctan(se / sn)

    # Calculate potential par ---------------------------------------------------

    # Step implemented in case azi is a single value i.e. azi = 39
    if isinstance(azi, float):
        # Transform float into array
        azi = np.array([azi])

    azi[sn > 0] = azi[sn > 0] + 3.1416

    pfd = solar_constant * exp(attenuation_coef / sin(alt))

    # Step implemented in case pfd is a single value i.e. pdf = 41
    if isinstance(pfd, float):
        # Transform float into array
        pfd = np.array([pfd])

    pfd[alt < 0] = 0

    dpfd = diffuse_fraction * pfd
    dpfd[alt < 0] = 0

    return dpfd + pfd * sin(alt)

#### __Example: Calculate the Potential PAR__

In [ ]:
potential_par(time_of_day_in_hours=19, latitude=45.1, day_of_year=100)

In [ ]:
potential_par(
    time_of_day_in_hours=np.arange(0, 24), latitude=45.1, day_of_year=100
)